In [11]:
# 이 코드는 데이터를 기준으로 두 개의 baseline 예측을 만들고, 미래 구간에서 RMSE와 MAE로 성능을 정량적으로 비교하는 코드다.

import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error

df = pd.read_csv("retail_store_inventory.csv")
df = df[(df["Region"] == "North") & (df["Category"] == "Groceries")].copy()

df["Date"] = pd.to_datetime(df["Date"]) #날짜 처리 + 시간 정렬
df = df.sort_values("Date") # 날자 기준으로 과거->미래 순서로 정렬

split_idx = int(len(df) * 0.8) # 전체 데이터의 80% 지점 인덱스 계산
train = df.iloc[:split_idx] # 과거 80% 학습용
test  = df.iloc[split_idx:] # 미래 20% 평가용

y_train = train["Units Sold"] # 기준값 계산용
y_test  = test["Units Sold"] # 오차 계산용


# RMSE 함수
def rmse(y_true, y_pred):
    return mean_squared_error(y_true, y_pred) ** 0.5  # <-- squared=False 대신 직접 sqrt

# Baseline 1: Historical mean
pred_mean = pd.Series([y_train.mean()] * len(test), index=test.index)
# Train 데이터의 Units Sold 평균값을 Test 데이터 길이만큼 복제해서 모든 날의 예측값으로 사용

# Baseline 2: Provided Demand Forecast
pred_forecast = test["Demand Forecast"]
# Test 구간에 있는 Demand Forecast 값을 그대로 예측값으로 사용

results = pd.DataFrame({
    "Model": ["Baseline 1 (Mean)", "Baseline 2 (Demand Forecast)"],
    "RMSE":  [rmse(y_test, pred_mean), rmse(y_test, pred_forecast)],
    "MAE":   [mean_absolute_error(y_test, pred_mean), mean_absolute_error(y_test, pred_forecast)]
})

print(results)

                          Model        RMSE        MAE
0             Baseline 1 (Mean)  101.097217  83.129556
1  Baseline 2 (Demand Forecast)   10.255921   8.599795


Two baseline models were evaluated. The historical mean baseline shows very poor predictive performance, while the provided demand forecast demonstrates strong accuracy with an RMSE of approximately 10 units. This demand forecast serves as a strong benchmark for evaluating more advanced models.

두 가지 기준선 모델을 평가하였다. 과거 평균을 사용한 baseline은 예측 성능이 매우 낮았으며, 데이터에 포함된 수요 예측값은 RMSE 약 10으로 높은 정확도를 보였다. 따라서 해당 수요 예측값을 이후 모델 성능 비교를 위한 주요 기준선으로 사용한다.